# PTAN库

宏观来讲，PTAN提供了下面的实体：Agent：知道如何将一批观察转换成一批需要执行的动作的类。

它还可以包含可选状态，当需要在一个片段中为后续动作记录一些信息的时候可以用到。

本库提供了好几个智能体用于最常见的一些RL场景，你也完全可以编写自己的BaseAgent子类。。

- ActionSelector：一小段与Agent协同工作的逻辑，它知道如何从网络的输出中选择动作。

- ExperienceSource和它的变体：Agent的实例和Gym环境对象可以提供关于片段轨迹的信息。它最简单的形式就是每次一个（a, r,s'）状态转移，但其功能远不止如此。


- ExperienceSourceBuffer和它的变体：具有各种特性的回放缓冲区。包含一个简单的回放缓冲区和两个版本的带优先级的回放缓冲区。各种工具类，比如TargetNet和用于时间序列预处理的包装器（用于在TensorBoard中追踪训练进度）。

- PyTorch Ignite帮助类可以将PTAN集成到Ignite框架中去。Gym环境包装器，例如Atari游戏的包装器（从OpenAI Baselines复制而来，并做了一些调整）。


## 动作选择器

- argmax：常被用在Q值方法中，也就是当用神经网络预测一组动作的Q值并需要一个Q(s, a)最大的动作时。

- 基于策略的：网络的输出是概率分布（以logits的形式或归一化分布的形式），并且动作需要从这个分布采样。第4章已提到过这种情况，也就是讨论交叉熵方法的时候。


In [134]:
import random
import gym

import numpy as np
import ptan
import torch.nn as nn

l = list(range(9))
random.shuffle(l)
q_vals = np.array(l).reshape(3, 3)
q_vals

array([[0, 1, 8],
       [2, 3, 7],
       [4, 5, 6]])


ArgmaxActionSelector：对传入张量的第二维执行argmax。（它假设参数是一个矩阵，并且它的第一维为批维度。作的概率。

In [16]:
selector = ptan.actions.ArgmaxActionSelector()
selector(q_vals)

array([0, 2, 2], dtype=int64)

ProbabilityActionSeletor：从离散动作集的概率分布中采样。



In [23]:
q_vals = nn.Softmax(dim=1)(torch.Tensor(q_vals))
q_vals

tensor([[0.5671, 0.2177, 0.2152],
        [0.2204, 0.5314, 0.2482],
        [0.4703, 0.2981, 0.2316]])

In [35]:
selector = ptan.actions.ProbabilityActionSelector()
selector(q_vals.numpy())

array([2, 1, 1])

EpsilonGreedyActionSelector：具有epsilon参数，用来指定选择随机动作的概率。

epsilon大小代表随机的概率，若为0，则没有采取随即动作.

In [67]:
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=0.2)
selector(q_vals.numpy())

array([0, 1, 0], dtype=int64)

## Agent

宏观来讲，智能体需要接受一批观察（以NumPy数组的形式）并返回一批它想执行的动作。

### DQNAgent
当动作空间不是非常大的时候，这个类可以适用于Q-learning，包括Atari游戏和很多经典的问题。

这个方法不是很通用，但本书的后面将介绍如何解决这个问题。

DQNAgent需要一批观察（NumPy数组）作为输入，使用网络来获得Q值，然后使用提供的ActionSelector将Q值转换成动作的索引。



In [73]:
class DQNNet(nn.Module):
    def __init__(self, obs_size, hidden_size, q_table_size):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, q_table_size),
        )

    def forward(self, state):
        return self.net(state)

In [114]:
net = DQNNet(4,16,2)
net(torch.rand(5,4))

tensor([[ 0.1239,  0.0239],
        [-0.0960,  0.1399],
        [ 0.0616,  0.0046],
        [ 0.0191,  0.1469],
        [-0.1176,  0.2227]], grad_fn=<AddmmBackward0>)

In [127]:
selector = ptan.actions.ArgmaxActionSelector()
agent = ptan.agent.DQNAgent(dqn_model=net, action_selector=selector)
agent(torch.rand(5,4))

(array([1, 1, 1, 1, 1], dtype=int64), [None, None, None, None, None])

### PolicyAgent

PolicyAgent需要神经网络生成离散动作集的策略分布。

策略分布可以是logits（未归一化的）分布，也可以是归一化分布。

实践中，最好都是用logits分布以提升训练过程的数值稳定性

In [131]:
a = torch.empty((5, 4)).uniform_(1,2)
a

tensor([[1.6077, 1.8959, 1.6269, 1.7592],
        [1.4544, 1.3852, 1.5736, 1.4986],
        [1.0109, 1.5741, 1.1709, 1.0381],
        [1.1970, 1.5519, 1.4666, 1.2554],
        [1.9007, 1.8573, 1.8272, 1.6637]])

In [132]:
selector = ptan.actions.ProbabilityActionSelector()
agent = ptan.agent.PolicyAgent(model=net, action_selector=selector,apply_softmax=True)
agent(a)

(array([0, 1, 1, 1, 1]), [None, None, None, None, None])

## 经验源

In [136]:
from typing import List, Optional, Tuple, Any

In [147]:
class ToyEnv(gym.Env):
    """
    Environment with observation 0..4 and actions 0..2
    Observations are rotated sequentialy mod 5, reward is equal to given action.
    Episodes are having fixed length of 10
    """

    def __init__(self):
        super(ToyEnv, self).__init__()
        self.observation_space = gym.spaces.Discrete(n=5)
        self.action_space = gym.spaces.Discrete(n=3)
        self.step_index = 0

    def reset(self):
        self.step_index = 0
        return self.step_index, {}

    def step(self, action):
        is_done = self.step_index == 10
        if is_done:
            return self.step_index % self.observation_space.n, \
                   0.0, is_done, {}
        self.step_index += 1
        return self.step_index % self.observation_space.n, \
               float(action), self.step_index == 10, False, {}

class DullAgent(ptan.agent.BaseAgent):
    """
    Agent always returns the fixed action
    """
    def __init__(self, action: int):
        self.action = action

    def __call__(self, observations: List[Any],
                 state: Optional[List] = None) \
            -> Tuple[List[int], Optional[List]]:
        return [self.action for _ in observations], state



In [153]:
env = ToyEnv()
agent = DullAgent(action=1)
exp_source = ptan.experience.ExperienceSource(env=env, agent=agent, steps_count=2)

In [155]:
for idx, exp in enumerate(exp_source):
    if idx > 15:
        break
    print(exp)

ValueError: too many values to unpack (expected 4)